In [1]:
import rospy
from geometry_msgs.msg import PoseStamped
from nav_msgs.msg import Odometry
from my_assignment_2.msg import PosVel
from assignment_2_2022.msg import PlanningActionResult

import actionlib # Bring in the SimpleActionClient

# Bring in the PlanningAction type, composed by the types PlanningGoal, PlanningResult and PlanningFeedback
from assignment_2_2022.msg import PlanningAction, PlanningGoal

# Since we'll publish within the callback function, we have to define the publisher globally
pub = rospy.Publisher("/pos_vel", PosVel, queue_size=10)

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

x_plot = []
y_plot = []
last_stamp = -1

%matplotlib widget

# First, we prepare the figure
np_x_plot = np.array(x_plot)
np_y_plot = np.array(y_plot)

fig, ax = plt.subplots() # Create a figure and an axes.
plt.axis([-10, 10, -10, 10])
plt.grid()
ax.set_xlabel('x') # Add an x-label to the axes.
ax.set_ylabel('y') # Add a y-label to the axes.
ax.set_title("Robot Position") # Add a title to the axes.

l, = plt.plot(np_x_plot, np_y_plot)

# ax.plot(np_x_plot, np_y_plot)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
def plotear():
#     plt.draw()
    fig.canvas.draw_idle()

In [4]:
def callback_odometry(odometry):
    
    my_pos_vel = PosVel() # this is the msg we'll publish
    
    # Fill the four fields relying on the data received via the topic /odom
    my_pos_vel.x = odometry.pose.pose.position.x
    my_pos_vel.y = odometry.pose.pose.position.y
    my_pos_vel.vel_x = odometry.twist.twist.linear.x
    my_pos_vel.vel_z = odometry.twist.twist.angular.z

    pub.publish(my_pos_vel) # publish
    
    global x_plot, y_plot, last_stamp
    
    if odometry.header.stamp.secs != last_stamp:
        
        x_plot.append(odometry.pose.pose.position.x)
        y_plot.append(odometry.pose.pose.position.y)
        
        np_x_plot = np.array(x_plot)
        np_y_plot = np.array(y_plot)
        
        l.set_xdata(np_x_plot)
        l.set_ydata(np_y_plot)
        
        plotear()
        
#         ax.plot(np_x_plot, np_y_plot)
        
        last_stamp = odometry.header.stamp.secs

In [5]:
# If the goal has been successfully reached (status=3), increment the parameter related to the number of goals reached
def callback_result(result):
    
    if result.status.status == 3:
        print("Goal reached!\n")
        global n_goals_reached
        n_goals_reached += 1
        rospy.set_param('goals_reached', n_goals_reached)

In [6]:
# Initialize a rospy node so that the SimpleActionClient can publish and subscribe over ROS
rospy.init_node('planning_client')

# Subscribe to the topics /odom and /reaching_goal/result
rospy.Subscriber("/odom", Odometry, callback_odometry)
rospy.Subscriber("/reaching_goal/result", PlanningActionResult, callback_result)

# Counters for the number of goals reached and cancelled, in total
n_goals_reached = 0
n_goals_cancelled = 0

# Initialize the parameters, in the ROS parameter service, related to these counters
rospy.set_param('goals_reached', n_goals_reached)
rospy.set_param('goals_cancelled', n_goals_cancelled)

# Create the SimpleActionClient. The service is called '/reaching_goal' and the type of action is 'PlanningAction'
client = actionlib.SimpleActionClient('reaching_goal', PlanningAction)

# Wait until the action server has started up and started listening for goals
print("Waiting for the action server to start...")
client.wait_for_server()
print("Action server up and running!\n")

Waiting for the action server to start...
Action server up and running!



In [7]:
import ipywidgets as widgets

# Get a new target from user input

print("To send a new target, specify the x and y coordinates and click \'Send\':\n")
print("To cancel the current target, click \'Cancel\':\n")
print("x = ")
x = widgets.FloatText()
display(x)
print("y = ")
y = widgets.FloatText()
display(y)

send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='success'
)

cancel_button = widgets.Button(
    description='Cancel',
    disabled=False,
    button_style='danger'
)

output = widgets.Output()

display(send_button, output)
display(cancel_button, output)

To send a new target, specify the x and y coordinates and click 'Send':

To cancel the current target, click 'Cancel':

x = 


FloatText(value=0.0)

y = 


FloatText(value=0.0)

Button(button_style='success', description='Send', style=ButtonStyle())

...

Goal (1.0, 1.0) sent
Heading to the goal...
Goal reached!

Goal cancelled



[ERROR] [1684491033.882573, 2819.361000]: Received comm state PREEMPTING when in simple state DONE with SimpleActionClient in NS /reaching_goal


Goal (1.0, 1.0) sent
Heading to the goal...
Goal (1.0, 1.0) sent
Heading to the goal...
Goal cancelled



[ERROR] [1684491055.516732, 2824.354000]: Received comm state PREEMPTING when in simple state DONE with SimpleActionClient in NS /reaching_goal


Goal (1.0, 1.0) sent
Heading to the goal...
Goal (1.0, 1.0) sent
Heading to the goal...
Goal (1.0, 1.0) sent
Heading to the goal...
Goal reached!

Goal (1.0, 1.0) sent
Heading to the goal...
Goal (1.0, 1.0) sent
Heading to the goal...


In [8]:
# If the user clicks the 'cancel' button, cancel the current goal and increment the parameter related to the number 
# of goals cancelled
def on_cancel_button_clicked(b):
        
    client.cancel_goal()
    print("Goal cancelled\n")
    global n_goals_cancelled
    n_goals_cancelled += 1
    rospy.set_param('goals_cancelled', n_goals_cancelled)

In [9]:
# If the user clicks the 'send' button...
def on_send_button_clicked(b):
        
    # Create a goal to send to the action server
    my_goal_pose = PoseStamped()
    my_goal_pose.pose.position.x = x.value
    my_goal_pose.pose.position.y = y.value
    goal = PlanningGoal(target_pose = my_goal_pose)

    # Send the goal to the action server
    client.send_goal(goal)

    print("Goal (" + str(x.value) + ", " + str(y.value) + ") sent")
    print("Heading to the goal...")

In [10]:
send_button.on_click(on_send_button_clicked)
cancel_button.on_click(on_cancel_button_clicked)

In [11]:
print("Distance to the closest obstacle: ")
box = widgets.FloatText(value=-1, disabled = True, layout = widgets.Layout(width='200px'))

display(box)

Distance to the closest obstacle: 


FloatText(value=-1.0, disabled=True, layout=Layout(width='200px'))

In [12]:
def callback_scan(msg):
    
    min_val = msg.range_max
    
    a = msg.ranges[0]
    
    for i in range(len(msg.ranges)):
        
        if (msg.ranges[i] < min_val) and (msg.ranges[i] >= msg.range_min) and (msg.ranges[i] <= msg.range_max):
            
            min_val = msg.ranges[i]
    
    global box
    box.value = min_val
            

In [13]:
from sensor_msgs.msg import LaserScan

# Subscribe to the topic /scan
s = rospy.Subscriber("/scan", LaserScan, callback_scan)